# Merging for validation on chromosome 1

Note/Warning for merging quality check for bed files merging

## Create validation file

In [ ]:
# import pandas as pd
# data = {
#     'chrom': ['chr12', 'chr15', 'chr16', 'chr20', 'chr20', 'chr7',],
#     'POS': [114704514, 90735064, 57302898, 1306044, 42780500, 4835971],
#     'POS_INCR': [114704515, 90735065, 57302899, 1306045, 42780501, 4835972]
# }

# # Create DataFrame from the dictionary
# df = pd.DataFrame(data)

# print(df)

   chrom        POS   POS_INCR
0  chr12  114704514  114704515
1  chr15   90735064   90735065
2  chr16   57302898   57302899
3  chr20    1306044    1306045
4  chr20   42780500   42780501
5   chr7    4835971    4835972


In [6]:
import pandas as pd
import numpy as np
chromosome = 'chr1'
df = pd.read_csv("Validation/homo_sapiens-{chromosome}.vcf", sep = '\t', skiprows = 36)
# df

In [3]:
df = df.rename({'#CHROM': 'chrom'}, axis = 1)
df = df.drop(['ID', 'FILTER', 'REF', 'ALT', 'QUAL', 'INFO'], axis = 1)
df = df.drop_duplicates(subset=["POS"])
df

KeyError: "['ID', 'FILTER', 'REF', 'ALT', 'QUAL', 'INFO'] not found in axis"

In [5]:
df['POS'].value_counts()

10001        1
171139709    1
171139665    1
171139667    1
171139676    1
            ..
76189878     1
76189882     1
76189883     1
76189890     1
248946420    1
Name: POS, Length: 53355185, dtype: int64

In [8]:
df.to_csv(f"Validation/validation_{chromosome}.txt", sep = '\t', index = False)

## Testing for validation file

In [1]:
# declare your variables here
import subprocess
import os 
import pandas as pd
chromosome ='chr1'
validation_file = f"Validation/validation_{chromosome}.txt"
logging_file = f"Validation/logging_{chromosome}"
from datetime import date

### ENCODE

#### Increment

In [9]:
import subprocess
chromosome ='chr1'
validation_file = f"Validation/validation_{chromosome}.txt"
validation = pd.read_csv(validation_file, sep = '\t')
validation['POS_INCR'] = validation['POS'] + 1
# Transform the 'chrom' column
# validation['chrom'] = validation['chrom'].apply(lambda x: f'chr{x}')

In [3]:
validation['POS'].value_counts()

10010        1
88563781     1
88563822     1
88563818     1
88563813     1
            ..
43734930     1
43734927     1
43734923     1
43734921     1
133265295    1
Name: POS, Length: 31176663, dtype: int64

In [10]:
validation.shape

(53355185, 3)

In [11]:
# validation['POS'].value_counts()
# save this validation into another txt file
validation.to_csv(f"Validation/validation_{chromosome}.txt", sep = '\t', index = False)

In [12]:
!wc "Validation/validation_chr1.txt"

  53355186  160065558 1122135084 Validation/validation_chr1.txt


In [18]:
os.listdir('/tljh-data/home/jupyter-ngocduy.tran/Validation merging test')

['chr12_atac_odd.txt',
 'output_even.txt',
 'chr12_atac_even.txt',
 'ch12_cvdkp_even.txt',
 'chr12_enc_tfbs_odd.txt',
 'chr1',
 'chr12_cvdkp_odd.txt',
 'output_odd.txt',
 'chr12_enc_tfbs_zebra_vista_even.txt',
 'chr12_encode_even.txt',
 'chr12_encode_odd.txt',
 'chr12_enc_tfbs_even.txt',
 '.ipynb_checkpoints',
 'chr12_enc_tfbs_zebra_vista_odd.txt']

In [22]:
os.chdir("/tljh-data/home/jupyter-ngocduy.tran/Validation merging test")
for i in range(1, 24):
    os.makedirs(f"chr{i}", exist_ok = True)
    

In [23]:
os.chdir("/tljh-data/home/jupyter-ngocduy.tran/")


#### ENCODE merging

In [ ]:
today = date.today()
import re

# create promoters txt files with: the first initial rows as the original dataframe
# while the most right column is the column produced by the bed file
def read_cres_files(mastersheet,file_lst, folder, subject, start_ind):
  output_odd = f"Validation merging test/{chromosome}/output_odd.txt"
  output_even = f"Validation merging test/{chromosome}/output_even.txt"
  track_ind = start_ind
  for bed_file in file_lst:
    try:

      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      with open(logging_file, 'w') as fpo:
          fpo.write(f"One missing file: {bed_file}\n")
      pass
    else:
      name = bed_file[: -4]
      print(name)
      # print(bed_file)
      # Run the Bedtools intersect command
      if track_ind == 0:
        output_file = output_even
        command = ['bedtools', 'intersect', '-C', '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      elif(track_ind %2 ==0):
        output_file = output_even
        command = ['bedtools', 'intersect', '-C', '-a', output_odd, '-b', os.path.join(folder, bed_file)]

      else:
        output_file=  output_odd
        command = ['bedtools', 'intersect', '-C', '-a', output_even, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)
      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)
      word_count = subprocess.run(["wc", output_file])
      print(word_count.stdout)
      overview = subprocess.run(["sed", "-n", "1p", output_file], capture_output = True, text = True)
      print(overview.stdout)

      # Display the output file name
      print(track_ind)
      print(f"Intersection saved as {name}")
      with open(logging_file, 'w') as fpo:
          fpo.write(name)
          # fpo.write(word_count.stdout)
          fpo.write(overview.stdout)
          fpo.write(str(track_ind))
      track_ind += 1

  return track_ind


def read_bed_files(mastersheet,subject, start_ind):
  folder = f"BED files/{subject}"
  embryo_folder = f"BED files/{subject}/Clean Embryo files"
  files = os.listdir(folder)

  promoters = sorted([ele for ele in files if 'promoter' in ele])
  enhancers = sorted([ele for ele in files if 'enhancer' in ele])
  # for mice, embryo is a redundant column as it is already either the
  # promtoer or enhancer
  if subject == 'Human':
    embryo = sorted([ele for ele in os.listdir(embryo_folder) if 'embryo' in ele])
  ctcf = sorted([ele for ele in files if 'CTCF' in ele])

  # quality check for Human: enough files generated
  # investigate_levels only applies to human dna only
  # investigate_levels(promoters, folder)
  print(promoters)
  ind = read_cres_files(mastersheet,promoters, folder, subject, start_ind = start_ind)
  print(f"Starting ind is: {ind}")
  # investigate_levels(enhancers, folder)
  print(enhancers)
  ind = read_cres_files(mastersheet,enhancers, folder, subject, start_ind = ind)
  print(f"Starting ind is: {ind}")

  # investigate_levels(embryo, embryo_folder)
  if subject == 'Human':
    print(embryo)
    ind = read_cres_files(mastersheet,embryo, embryo_folder, subject, start_ind = ind)
    print(f"Starting ind is: {ind}")
  print(ctcf)
  # investigate_levels(ctcf, folder)
  ind = read_cres_files(mastersheet,ctcf, folder, subject, start_ind = ind)
  print(f"Starting ind is: {ind}")

  print(len(promoters), len(enhancers), len(ctcf))
  return ind

bed_starting_ind = read_bed_files(validation_file, subject = "Human", start_ind = 0)
print(f"bed_starting_ind: {bed_starting_ind}")
bed_starting_ind = read_bed_files(validation_file, subject = 'Mouse', start_ind = bed_starting_ind)


['heart_left_ventricle_tissue_female_adult_53_years.promoters.bed', 'heart_left_ventricle_tissue_male_adult_34_years.promoters.bed', 'heart_left_ventricle_tissue_male_child_3_years.promoters.bed', 'heart_right_ventricle_tissue_female_adult_46_years.promoters.bed', 'heart_right_ventricle_tissue_male_adult_34_years.promoters.bed', 'heart_right_ventricle_tissue_male_adult_40_years.promoters.bed', 'heart_right_ventricle_tissue_male_child_3_years.promoters.bed']
heart_left_ventricle_tissue_female_adult_53_years.promoters
  53355185  213420740 1228845435 Validation merging test/chr1/output_even.txt
None
1	10001	10002	0

0
Intersection saved as heart_left_ventricle_tissue_female_adult_53_years.promoters
One missing file: heart_left_ventricle_tissue_male_adult_34_years.promoters.bed
One missing file: heart_left_ventricle_tissue_male_child_3_years.promoters.bed
heart_right_ventricle_tissue_female_adult_46_years.promoters
  53355185  266775925 1335555805 Validation merging test/chr1/output_odd.t

In [3]:
bed_starting_ind = 42
print(bed_starting_ind)
# order: Human - Mouse - TFBS - zebrafish - vista - cvdkp - hic - pchic -atac - genomic (fetal) - ncer - dickel

42


### TFBS

In [4]:
import os
import subprocess
import numpy as np 
import pandas as pd
from datetime import date
today = date.today()
import re

def read_cres_files(mastersheet,file_lst, folder, start_ind, intersect):
  output_odd = f"Validation merging test/{chromosome}/output_odd.txt"
  output_even = f"Validation merging test/{chromosome}/output_even.txt"
  track_ind = start_ind
  for bed_file in file_lst:
    try:

      df = pd.read_csv(os.path.join(folder, bed_file), sep = '\t', header = None)
    except pd.errors.EmptyDataError:
      # handle empty file
      print(f"One missing file: {bed_file}")
      pass
    else:
      name = bed_file[: -4]
      print(name)
      # print(bed_file)
      # Run the Bedtools intersect command
      if track_ind == 0:
        output_file = output_even
        command = ['bedtools', 'intersect', intersect, '-a', mastersheet, '-b', os.path.join(folder, bed_file)]
      elif(track_ind %2 ==0):
        output_file = output_even
        command = ['bedtools', 'intersect', intersect, '-a', output_odd, '-b', os.path.join(folder, bed_file)]

      else:
        output_file=  output_odd
        command = ['bedtools', 'intersect', intersect, '-a', output_even, '-b', os.path.join(folder, bed_file)]
      result = subprocess.run(command, capture_output=True, text=True)
      # Save the intersected result to the output file
      with open(output_file, 'w') as f:
          f.write(result.stdout)

      word_count = subprocess.run(["wc", output_file])
      overview = subprocess.run(["sed", "-n", "1p", output_file], capture_output = True, text = True)
      print(overview.stdout)
      with open(logging_file, 'a') as fpo:
          fpo.write(name)
          fpo.write("\n")
          fpo.write(overview.stdout)
          fpo.write("\n")
          fpo.write(str(track_ind))
          fpo.write("\n")
          

      # Display the output file name
      print(f"Intersection saved as {name}")
      track_ind += 1

  return track_ind

# example of subfolder: /hg38
def read_general_files(mastersheet,subject, start_ind, sub_folder = "", intersect = '-C'):
  folder = f"BED files/{subject}{sub_folder}"

  if subject in ['CVDKP', 'Hi-C']:
      files = [ele for ele in sorted(os.listdir(folder)) if 'unmapped' not in ele]
  elif subject in ['TFBS']:
      files = [ele for ele in sorted(os.listdir(folder)) if 'bed' in ele]
      
  else:    
      files = [ele for ele in sorted(os.listdir(folder))]

  print(len(files), files)

  ind = read_cres_files(mastersheet,files, folder, start_ind = start_ind, intersect = intersect)
  return ind
# cvdkp and hi-c requires unmapped
bed_starting_ind_tfbs = read_general_files(validation_file, "TFBS", start_ind = bed_starting_ind, intersect = '-C')
# zebrafish is good, vista requires sub_folder, cvdkp requires subfolder, hic is fine, pchic is fine, atac is fine, genomic analysis is fine 
# bed_starting_ind_zebra = read_general_files(validation_file, "Zebrafish", start_ind = bed_starting_ind_tfbs, intersect = '-C')

76 ['ASCL1_pwmscan_hg38_29347_22951.bed', 'CREB1_pwmscan_hg38_7806_15552.bed', 'EOMES_pwmscan_hg38_45327_5339.bed', 'FOXC1_pwmscan_hg38_14051_14776.bed', 'FOXC2_pwmscan_hg38_30036_45203.bed', 'FOXF1_pwmscan_hg38_28598_34204.bed', 'FOXH1_pwmscan_hg38_43315_3114.bed', 'FOXL1_pwmscan_hg38_43316_13581.bed', 'GATA3_pwmscan_hg38_11738_12391.bed', 'GATA4_pwmscan_hg38_28596_35342.bed', 'GATA5_pwmscan_hg38_9066_18689.bed', 'GATA6_pwmscan_hg38_20055_21663.bed', 'GLI1_pwmscan_hg38_33818_38538.bed', 'GLI2_pwmscan_hg38_7492_29088.bed', 'GLI3_pwmscan_hg38_35563_38491.bed', 'GRHL2_pwmscan_hg38_29347_19788.bed', 'HAND1_pwmscan_hg38_38445_41025.bed', 'HES1_pwmscan_hg38_35815_41265.bed', 'HEY1_pwmscan_hg38_28887_46496.bed', 'HEY2_pwmscan_hg38_43317_44258.bed', 'HIF1A_pwmscan_hg38_15529_13200.bed', 'ISL1_pwmscan_hg38_35815_41742.bed', 'JUN_pwmscan_hg38_38048_1878.bed', 'LBX1_pwmscan_hg38_27664_45938.bed', 'MEF2A_pwmscan_hg38_7806_11341.bed', 'MEF2C_pwmscan_hg38_43317_529.bed', 'MEF2D_pwmscan_hg38_11121_1

In [5]:
print(bed_starting_ind_tfbs) # bed_starting_ind_tfbs = 118

118


In [ ]:
# bed_starting_ind_zebra = read_general_files(validation_file, "Zebrafish", start_ind = bed_starting_ind_tfbs, intersect = '-C')
# print(bed_starting_ind_zebra)
# remove this feature 

In [6]:
bed_starting_ind_vista = read_general_files(validation_file, "Vista_human_heart_enhancers", start_ind = bed_starting_ind_tfbs, 
                                            intersect = "-C", sub_folder = "/hg38")

1 ['vista_hg38.bed']
vista_hg38
   53355185  6509332570 13820669095 Validation merging test/chr1/output_even.txt
1	10001	10002	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0

Intersection saved as vista_hg38


In [7]:
print(bed_starting_ind_vista)

119


In [8]:
# order: Human - Mouse - TFBS - zebrafish - vista - cvdkp - hic - pchic -atac - genomic (fetal) - ncer - dickel
# bed_starting_ind_zebra = read_general_files(validation_file, "Zebrafish", start_ind = bed_starting_ind_tfbs, intersect = '-C')
# print(bed_starting_ind_zebra)
# bed_starting_ind_vista = read_general_files(validation_file, "Vista_human_heart_enhancers", start_ind = bed_starting_ind_zebra, 
                                            # intersect = "-C", sub_folder = "/hg38")
bed_starting_ind_cvdkp = read_general_files(validation_file, "CVDKP", start_ind = bed_starting_ind_vista, 
                                            intersect = "-C", sub_folder = "/gz lifted bed files")
print(bed_starting_ind_cvdkp)
# bed_starting_ind_hic = read_general_files(validation_file, "Hi-C", start_ind = bed_starting_ind_cvdkp, intersect = '-C')
# bed_starting_ind_pchic = read_general_files(validation_file, "PCHiC", start_ind = bed_starting_ind_hic, intersect = '-C')   
# bed_starting_ind_atac = read_general_files(validation_file, "ATAC", start_ind = bed_starting_ind_pchic, intersect = '-C')
# bed_starting_ind_fetal = read_general_files(validation_file, "Genomic analyses clean fetal heart enhancers", start_ind = bed_starting_ind_atac)

30 ['DFF012AGB.bed', 'DFF017NNG.bed', 'DFF029APW.bed', 'DFF090AWG.bed', 'DFF165PSK.bed', 'DFF201AVG.bed', 'DFF208QZK.bed', 'DFF233YXQ.bed', 'DFF249RGM.bed', 'DFF262NPA.bed', 'DFF307JWU.bed', 'DFF307LLE.bed', 'DFF316NIT.bed', 'DFF367ZAV.bed', 'DFF371HWB.bed', 'DFF444FXW.bed', 'DFF491GMJ.bed', 'DFF512TCX.bed', 'DFF528FIC.bed', 'DFF552AGR.bed', 'DFF559UZT.bed', 'DFF564GOU.bed', 'DFF565QGM.bed', 'DFF621YNV.bed', 'DFF653TBJ.bed', 'DFF729PKJ.bed', 'DFF803PYP.bed', 'DFF842KEC.bed', 'DFF923IAT.bed', 'DFF975AHB.bed']
DFF012AGB
   53355185  6562687755 13927379465 Validation merging test/chr1/output_odd.txt
1	10001	10002	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0

Intersection saved as DFF012AGB
DFF017NNG
   53355185  6616042940 14034089835 Validation merging test/chr1/output_even.txt
1	10001	10002	0	0	

In [ ]:
print(bed_starting_ind_cvdkp) #150
bed_starting_ind_atac = read_general_files(validation_file, "ATAC", start_ind = bed_starting_ind_cvdkp, intersect = '-C')
# bed_starting_ind_atac = 188

149
38 ['ENCFF024FSS.bed', 'ENCFF055YGM.bed', 'ENCFF072LFQ.bed', 'ENCFF124JSN.bed', 'ENCFF174HPZ.bed', 'ENCFF179JFB.bed', 'ENCFF199VHV.bed', 'ENCFF237JBZ.bed', 'ENCFF239PZJ.bed', 'ENCFF251ADS.bed', 'ENCFF257UAY.bed', 'ENCFF258HEB.bed', 'ENCFF291YRH.bed', 'ENCFF303AKJ.bed', 'ENCFF315UYG.bed', 'ENCFF372VNL.bed', 'ENCFF525ZRG.bed', 'ENCFF529QEX.bed', 'ENCFF556YGF.bed', 'ENCFF560EBG.bed', 'ENCFF567VUC.bed', 'ENCFF643QLU.bed', 'ENCFF718BTP.bed', 'ENCFF720GJU.bed', 'ENCFF731JBR.bed', 'ENCFF783MYY.bed', 'ENCFF792EIF.bed', 'ENCFF811VNG.bed', 'ENCFF820DKN.bed', 'ENCFF830CEC.bed', 'ENCFF890NRK.bed', 'ENCFF898WYV.bed', 'ENCFF901EKY.bed', 'ENCFF953FJE.bed', 'ENCFF968YWL.bed', 'ENCFF984UBD.bed', 'ENCFF988KSK.bed', 'ENCFF991MKU.bed']
ENCFF024FSS
   53355185  8163343305 17128690565 Validation merging test/chr1/output_odd.txt
1	10001	10002	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	

In [ ]:
print(bed_starting_ind_atac)
bed_starting_ind_hic = read_general_files(validation_file, "Hi-C", start_ind = bed_starting_ind_atac, intersect = '-C')
bed_starting_ind_pchic = read_general_files(validation_file, "PCHiC", start_ind = bed_starting_ind_hic, intersect = '-C')  

In [ ]:
print(bed_starting_ind_pchic)
bed_starting_ind_fetal = read_general_files(validation_file, "Genomic analyses clean fetal heart enhancers", start_ind = bed_starting_ind_pchic)
print(bed_starting_ind_fetal)

In [ ]:
# for dickel and ncer
# run bedtools intersection with the chr12_encode_even.txt with ncer and dickel then use paste -d'\t' to append the last column to the dataset